In [2]:
from atproto import FirehoseSubscribeReposClient, parse_subscribe_repos_message




In [39]:
import json
from atproto import CAR
from atproto_client.models.utils import get_or_create
from atproto_client import models

class JSONExtra(json.JSONEncoder):
    """raw objects sometimes contain CID() objects, which
    seem to be references to something elsewhere in bluesky.
    So, we 'serialise' these as a string representation,
    which is a hack but whatevAAAAR"""
    def default(self, obj):
        try:
            result = json.JSONEncoder.default(self, obj)
            return result
        except:
            return repr(obj)


client = FirehoseSubscribeReposClient()

test_list = []
test_op = []

def on_message_handler(message):
    global test_list
    commit = parse_subscribe_repos_message(message)
    if not isinstance(commit, models.ComAtprotoSyncSubscribeRepos.Commit):
        return
    car = CAR.from_bytes(commit.blocks)
    for op in commit.ops:
        if op.action in ["create"] and op.cid:
            raw = car.blocks.get(op.cid)
            cooked = get_or_create(raw, strict=False)
            test_list.append(cooked)
            test_op.append(op)
            if len(test_list) > 5:
                client.stop()
   
client.start(on_message_handler)

In [37]:
record = test_list[101]
record

Record(created_at='2024-12-04T23:04:52.286Z', text='💙', embed=None, entities=None, facets=None, labels=None, langs=['en'], reply=ReplyRef(parent=Main(cid='bafyreigzuvcoqwrueo2idpc4kyarryhd74fmuwx7h7cmjzkroa2xq6kklq', uri='at://did:plc:oqqeaqhjkgyde2u7b6vmxfj4/app.bsky.feed.post/3lcjb6xb2x225', py_type='com.atproto.repo.strongRef'), root=Main(cid='bafyreifdiwofzzrrwhqr2yj7t24h5dby56n7uh3ih54tg5tgxs5sxraucu', uri='at://did:plc:bb34nndauqsl2rdtfkq5jtda/app.bsky.feed.post/3lcjaw7n7vc2z', py_type='com.atproto.repo.strongRef'), py_type='app.bsky.feed.post#replyRef'), tags=None, py_type='app.bsky.feed.post')

In [41]:
op = test_op[1]

In [42]:
op

RepoOp(action='create', path='app.bsky.feed.post/3lcjbuoh6wc24', cid=CID(version=1, codec=113, hash=Multihash(code=18, size=32, digest=b'\x84\x04P\xf8\xf9^6\xe7\xec\xe4\xc7\x97R\xaf\x0e\x99\xfe\xf4j\xc0\xab]KZX\xfca=t\x9a\x13\x11'), _stringified_form='bafyreieearipr6k6g3t6zzghs5jk6duz732gvqfllvfvuwh4me6xjgqtce', _raw_byte_form=b'\x01q\x12 \x84\x04P\xf8\xf9^6\xe7\xec\xe4\xc7\x97R\xaf\x0e\x99\xfe\xf4j\xc0\xab]KZX\xfca=t\x9a\x13\x11'), py_type='com.atproto.sync.subscribeRepos#repoOp')

In [69]:
## websocket approach

import websockets
import asyncio
import json

# fix asyncio.run() to run the function in notebook


did_list = []
commit_list = []
async def connect_jetstream():
    uri = "wss://jetstream2.us-east.bsky.network/subscribe"
    params = {
        "wantedCollections": ["app.bsky.feed.post"]  # Only get posts
    }
    
    # Build URL with parameters
    url = f"{uri}?{'&'.join(f'wantedCollections={c}' for c in params['wantedCollections'])}"
    
    counter = 0
    
    async with websockets.connect(url) as websocket:
        while True:
            try:
                message = await websocket.recv()
                data = json.loads(message)
                
                # Process different kinds of messages
                if data["kind"] == "commit":
                    commit = data["commit"]
                    commit_list.append(commit)
                    if commit["operation"] == "create":
                        print(f"New post from {data['did']}")
                        print(f"Content: {commit['record']}")
                        print("---")
                        counter += 1
                        did_list.append(data['did'])
                        if counter > 10:
                            break
                elif data["kind"] == "identity":
                    print(f"Identity update for {data['identity']['did']}")
                    print(f"New handle: {data['identity']['handle']}")
                    print("---")
                
            except websockets.exceptions.ConnectionClosed:
                print("Connection closed, attempting to reconnect...")
                break

# Run the async function in a way that works in notebook
await connect_jetstream()


New post from did:plc:kldsvs34gh5z6zkp6w4vctwn
Content: {'$type': 'app.bsky.feed.post', 'createdAt': '2024-12-04T23:54:40.184Z', 'embed': {'$type': 'app.bsky.embed.record', 'record': {'cid': 'bafyreihqpkfessajn53k6astzvl2mwdgldle3yvo36m4eomo7vfgj4fyzi', 'uri': 'at://did:plc:fqfrgk62jxqpo3muso24c7dn/app.bsky.feed.post/3lcjd7vmsyk2g'}}, 'langs': ['en'], 'text': 'Very much this 👇.'}
---
New post from did:plc:kth4rcicjamgikqq2ttbudha
Content: {'$type': 'app.bsky.feed.post', 'createdAt': '2024-12-04T23:54:40.235Z', 'langs': ['pt'], 'text': 'atualizações: esses semestre sugou totalmente minhas energias, minha saúde mental e minha saúde física eu só tenho vontade de brigar com algumas pessoas (merecem levar um fecho), GRITAR e chorar (não tenho tempo)'}
---
New post from did:plc:f6vf6dp6xzzw3s5hdvuaraum
Content: {'$type': 'app.bsky.feed.post', 'createdAt': '2024-12-04T23:54:40.035Z', 'langs': ['en'], 'text': 'Unhnh hnh body writing.....'}
---
New post from did:plc:jf5w6g3anh7av4erpqnm47nz
Con

In [55]:
response = normal_client.com.atproto.repo.describe_repo({'repo': did_list[3]})
response

Response(collections=['app.bsky.actor.profile', 'app.bsky.feed.like', 'app.bsky.feed.post', 'app.bsky.feed.repost', 'app.bsky.graph.follow'], did='did:plc:jje32a6o5kzc3vos5dm5bxn6', did_doc={'@context': ['https://www.w3.org/ns/did/v1', 'https://w3id.org/security/multikey/v1', 'https://w3id.org/security/suites/secp256k1-2019/v1'], 'id': 'did:plc:jje32a6o5kzc3vos5dm5bxn6', 'alsoKnownAs': ['at://antijunk.bsky.social'], 'verificationMethod': [{'id': 'did:plc:jje32a6o5kzc3vos5dm5bxn6#atproto', 'type': 'Multikey', 'controller': 'did:plc:jje32a6o5kzc3vos5dm5bxn6', 'publicKeyMultibase': 'zQ3shPMCyeu1HYAi85JVTmapfJhPjAoH57PgN2R9FYpcoWx4Y'}], 'service': [{'id': '#atproto_pds', 'type': 'AtprotoPersonalDataServer', 'serviceEndpoint': 'https://gomphus.us-west.host.bsky.network'}]}, handle='antijunk.bsky.social', handle_is_correct=True)

In [57]:
from atproto import DidDocument
did_doc = DidDocument.from_dict(response.did_doc)

In [67]:
normal_client = Client()

normal_client.get_post("app.bsky.feed.post/3lcj5epci6c2n")


LoginRequiredError: To perform this action, you must be logged in. Use the `login` method first.

In [73]:
commit_list[5]

{'rev': '3lcje2p3j7p2p',
 'type': 'c',
 'operation': 'create',
 'collection': 'app.bsky.feed.post',
 'rkey': '3lcje2lspks2n',
 'record': {'$type': 'app.bsky.feed.post',
  'createdAt': '2024-12-04T23:54:37.504Z',
  'langs': ['en'],
  'reply': {'parent': {'cid': 'bafyreie6noeildxt6lzopaa3skaxehlqozyrnrynqmcyegamxm6vlsd6ue',
    'uri': 'at://did:plc:7upoypvlxudsiv4pzbi5jtwf/app.bsky.feed.post/3lcjdzi33uc2m'},
   'root': {'cid': 'bafyreie6noeildxt6lzopaa3skaxehlqozyrnrynqmcyegamxm6vlsd6ue',
    'uri': 'at://did:plc:7upoypvlxudsiv4pzbi5jtwf/app.bsky.feed.post/3lcjdzi33uc2m'}},
  'text': 'Funny how that always seems to happen.'},
 'cid': 'bafyreiguqtzbtnpee6xvyiit7sdzub4gz7c77rawqcxl64g32bmo7g626a'}